# 개발환경


######운영체제 : window 64 bit  
######언어 : 파이썬  
######라이브러리 버젼 :

*   Python - 3.7.14
*   pandas - 1.3.5
*   numpy - 1.21.5
*   xgboost - 0.90
*   scikit-learn - 1.0.2
*   tqdm - 4.64.1












패키지 로드

In [6]:
! pip install xgboost==0.90
! pip install numpy==1.21.5
! pip install pandas==1.3.5
! pip install scikit-learn==1.0.2
! pip install datetime
! pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from tqdm import tqdm
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from datetime import datetime, timedelta
import copy
import zipfile

날짜를 시간을 제외하고 년도월일 형태로 변환하는 함수

In [ ]:
def change_year(date):
    date = date[:10].replace("-","")
    return date

장비들이 작동하고 있는 시간만 추출하는 함수

In [ ]:
def filter_pump(x):
    return x[x['펌프작동남은시간']!=0]

def filter_cold(x):
    return x[x['냉방작동남은시간']!=0]

def filter_hot(x):
    return x[x['난방작동남은시간']!=0]

def filter_inside_air(x):
    return x[x['내부유동팬작동남은시간']!=0]

def filter_outside_air(x):
    return x[x['외부환기팬작동남은시간']!=0]

def filter_white(x):
    return x[x['화이트 LED작동남은시간']!=0]

def filter_blue(x):
    return x[x['블루 LED작동남은시간']!=0]

def filter_red(x):
    return x[x['레드 LED작동남은시간']!=0]

def filter_total_red(x):
    return x[x['적색광추정광량']!=0]

def filter_total_blue(x):
    return x[x['청색광추정광량']!=0]

def filter_total_white(x):
    return x[x['백색광추정광량']!=0]

def is_in(type_list, type_dict, i):
    try:
        type_list.append(type_dict[i])
    except:
        type_list.append(0)

컬럼 추가/삭제 함수

In [ ]:
def change_col(particular_case):
    particular_case['시간'] = particular_case['시간'].apply(change_year)
    day_list = list(particular_case['시간'].drop_duplicates(keep='first'))
    group_df = particular_case.groupby('시간')
    
    pump_list = []
    group_pump = group_df.apply(filter_pump)
    pump_dict = group_pump['시간'].value_counts().to_dict()
    
    cold_list = []
    group_cold = group_df.apply(filter_cold)
    cold_dict = group_cold['시간'].value_counts().to_dict()
     
    hot_list = []
    group_hot = group_df.apply(filter_hot)
    hot_dict = group_hot['시간'].value_counts().to_dict()
    
    inside_list = []
    group_inside = group_df.apply(filter_inside_air)
    inside_dict = group_inside['시간'].value_counts().to_dict()
    
    outside_list = []
    group_outside = group_df.apply(filter_outside_air)
    outside_dict = group_outside['시간'].value_counts().to_dict()
    
    white_list = []
    group_white = group_df.apply(filter_white)
    white_dict = group_white['시간'].value_counts().to_dict()
    
    blue_list = []
    group_blue = group_df.apply(filter_blue)
    blue_dict = group_blue['시간'].value_counts().to_dict()
    
    red_list = []
    group_red = group_df.apply(filter_red)
    red_dict = group_red['시간'].value_counts().to_dict()
    
    total_red_list = []
    group_total_red = group_df.apply(filter_total_red)
    group_total_red = group_total_red.reset_index(drop=True)
    group_total_red = group_total_red.groupby('시간')
    total_red_dict = (group_total_red['적색광추정광량'].max() - group_total_red['적색광추정광량'].min()).dropna().to_dict()
    
    max_red = []
    max_red_dict = group_total_red['적색광추정광량'].max().to_dict()
    min_red = []
    min_red_dict = group_total_red['적색광추정광량'].min().to_dict()
    
    total_blue_list = []
    group_total_blue = group_df.apply(filter_total_blue)
    group_total_blue = group_total_blue.reset_index(drop=True)
    group_total_blue = group_total_blue.groupby('시간')
    total_blue_dict = (group_total_blue['청색광추정광량'].max() - group_total_blue['청색광추정광량'].min()).dropna().to_dict()
    
    max_blue = []
    max_blue_dict = group_total_blue['청색광추정광량'].max().to_dict()
    min_blue = []
    min_blue_dict = group_total_blue['청색광추정광량'].min().to_dict()
    
    total_white_list = []
    group_total_white = group_df.apply(filter_total_white)
    group_total_white = group_total_white.reset_index(drop=True)
    group_total_white = group_total_white.groupby('시간')
    total_white_dict = (group_total_white['백색광추정광량'].max() - group_total_white['백색광추정광량'].min()).dropna().to_dict()
    
    max_white = []
    max_white_dict = group_total_white['백색광추정광량'].max().to_dict()
    min_white = []
    min_white_dict = group_total_white['백색광추정광량'].min().to_dict()
    
    sleep_list = []
    sleep_time = particular_case[(particular_case['화이트 LED작동남은시간']==0)&(particular_case['레드 LED작동남은시간']==0)&(particular_case['블루 LED작동남은시간']==0)]
    sleep_time = sleep_time.groupby('시간')
    sleep_time = sleep_time.size().to_dict()
    
    for i in day_list:
        is_in(sleep_list, sleep_time, i)
        is_in(pump_list, pump_dict, i)
        is_in(cold_list, cold_dict, i)
        is_in(hot_list, hot_dict, i)
        is_in(inside_list, inside_dict, i)
        is_in(outside_list, outside_dict, i)
        is_in(white_list, white_dict, i)
        is_in(blue_list, blue_dict, i)
        is_in(red_list, red_dict, i)
        is_in(total_red_list, total_red_dict, i)
        is_in(total_blue_list, total_blue_dict, i)
        is_in(total_white_list, total_white_dict, i)
        
        is_in(max_red, max_red_dict, i)
        is_in(min_red, min_red_dict, i)
        is_in(max_blue, max_blue_dict, i)
        is_in(min_blue, min_blue_dict, i)
        is_in(max_white, max_white_dict, i)
        is_in(min_white, min_white_dict, i)
            
    new_group_df = group_df.mean()
    
    new_group_df['냉방 시간'] = cold_list
    new_group_df['난방 시간'] = hot_list
    new_group_df['내부 팬'] = inside_list
    new_group_df['외부 팬'] = outside_list
    new_group_df['백색'] = white_list
    new_group_df['청색'] = blue_list
    new_group_df['적색'] = red_list
    new_group_df['LED_off'] = sleep_list
    new_group_df['적색 크기차'] = total_red_list
    new_group_df['청색 크기차'] = total_blue_list
    new_group_df['백색 크기차'] = total_white_list
    
    new_group_df['적색 최대'] = max_red
    new_group_df['적색 최소'] = min_red
    new_group_df['청색 최대'] = max_blue
    new_group_df['청색 최소'] = min_blue
    new_group_df['백색 최대'] = max_white
    new_group_df['백색 최소'] = min_white

    new_group_df['펌프 작동시간'] = pump_list
    new_group_df['실내 일교차'] = list(group_df['내부온도관측치'].max() - group_df['내부온도관측치'].min())
    new_group_df['습도_온도'] = new_group_df['내부온도관측치'] * new_group_df['내부습도관측치']
    new_group_df['습도_온도_CO2'] = new_group_df['습도_온도'] * new_group_df['CO2관측치']
    return new_group_df
   

train, test set 생성 함수

In [ ]:
def get_data(data_list):
    copy_data_list = copy.deepcopy(data_list)
    new_data = ""
    for i in range(len(copy_data_list)):
        if i == 0:
            new_data = copy_data_list[i]
        else:
            new_data = pd.concat([new_data, copy_data_list[i]])
    del_list = []
    for i in new_data.columns:
        if '남은시간' in i:
            del_list.append(i)
    new_data = new_data.drop(del_list, axis=1)
    return new_data

교차 검증 함수

In [ ]:
def test_(df):
    test_train = df.copy(deep=True)
    test_train['label'] = np.log1p(test_train['label'])

    y_train = test_train['label']
    X = test_train.drop(['label'],axis=1)
    X_col = X.columns
    scaler = StandardScaler()
    scaler.fit(X)
    scaled_X_train  = scaler.transform(X)
    scaled_X_train  = pd.DataFrame(scaled_X_train , columns=X_col)

    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', max_depth=5)
    nmse = cross_validate(xg_reg, scaled_X_train, y_train,
                      scoring=['neg_mean_squared_error'],
                      return_train_score=True,
                      cv=5, n_jobs=-1)
    mse = -1 * nmse['test_neg_mean_squared_error']
    rmse = np.sqrt(mse)
    avg_rmse = round(np.mean(rmse),4)
    print(f"평균 RMSE :{avg_rmse :.4f}")
    return avg_rmse

새로운 컬럼 삽입 함수

In [ ]:
def get_new_col(df, new_col):
  cal_train = df.copy(deep=True)
  for i in new_col:
      split_col = i.split('*')
      if len(split_col) >= 2:
          col_name = ""
          for j in range(len(split_col)):
              if j == 0:
                  col_name = split_col[j]
                  cal_train['cal'] = cal_train[split_col[j]]
              else:
                  col_name += f'*{split_col[j]}'
                  cal_train['cal'] *= cal_train[split_col[j]]
          df[col_name] = cal_train['cal']
  return df

train_list 생성

In [ ]:
train_list = []
col_list = []
for case in tqdm(range(1, 59)):
    if case < 10:
        case = '0'+str(case)
    particular_case = pd.read_csv(f'data/train_input/CASE_{case}.csv').interpolate()
    group_ = change_col(particular_case)
    particular_label = pd.read_csv(f'data/train_target/CASE_{case}.csv').interpolate()
    group_ = group_.reset_index()
    date_list = []
    for date in particular_label['시간'].apply(change_year):
      date = datetime.strptime(str(date), "%Y%m%d")
      before_day = date - timedelta(days=1)
      date_list.append(before_day.strftime('%Y%m%d'))
    group_ = group_[group_['시간'].isin(date_list)]
    group_ = group_.sort_values(by='시간')
    if case =='01': # 컬럼 명 수정
        col_list = group_.columns
    else:
        group_.columns = col_list
    group_['label'] = list(particular_label['rate'])
    growth_day = []
    for j in range(len(group_)):
      growth_day.append(j+1)
    group_['growth_day'] = growth_day
    group_ = group_.drop(['시간', '기준온도', '외부온도관측치','외부습도관측치', '카메라상태'], axis=1)
    train_list.append(group_)

100%|██████████| 58/58 [00:33<00:00,  1.75it/s]


In [ ]:
test_list = []
col_list = []
for case in tqdm(range(1, 7)):
    if case < 10:
        case = '0'+str(case)
    particular_case = pd.read_csv(f'data/test_input/TEST_{case}.csv').interpolate()
    group_ = change_col(particular_case)    
    particular_label = pd.read_csv(f'data/test_target/TEST_{case}.csv').interpolate()
    group_ = group_.reset_index()
    date_list = []
    for date in particular_label['시간'].apply(change_year):
      date = datetime.strptime(str(date), "%Y%m%d")
      before_day = date - timedelta(days=1)
      date_list.append(before_day.strftime('%Y%m%d'))
    group_ = group_[group_['시간'].isin(date_list)]
    group_ = group_.sort_values(by='시간')
    if case =='01': # 컬럼 명 수정
        col_list = group_.columns
    else:
        group_.columns = col_list
    group_['label'] = list(particular_label['rate'])
    growth_day = []
    for j in range(len(group_)):
      growth_day.append(j+1)
    group_['growth_day'] = growth_day
    group_ = group_.drop(['시간', '기준온도', '외부온도관측치','외부습도관측치', '카메라상태'], axis=1)
    test_list.append(group_)

100%|██████████| 6/6 [00:03<00:00,  1.67it/s]


train set, test set 생성

In [ ]:
train = get_data(train_list)
test = get_data(test_list)

변수 컬럼 추가

In [ ]:
train.columns

Index(['내부온도관측치', '내부습도관측치', 'CO2관측치', 'EC관측치', '펌프상태', '최근분무량', '일간누적분무량',
       '냉방상태', '난방상태', '내부유동팬상태', '외부환기팬상태', '화이트 LED상태', '화이트 LED동작강도',
       '레드 LED상태', '레드 LED동작강도', '블루 LED상태', '블루 LED동작강도', '냉방온도', '난방온도',
       '난방부하', '냉방부하', '총추정광량', '백색광추정광량', '적색광추정광량', '청색광추정광량', '냉방 시간',
       '난방 시간', '내부 팬', '외부 팬', '백색', '청색', '적색', 'LED_off', '적색 크기차',
       '청색 크기차', '백색 크기차', '적색 최대', '적색 최소', '청색 최대', '청색 최소', '백색 최대',
       '백색 최소', '펌프 작동시간', '실내 일교차', '습도_온도', '습도_온도_CO2', 'label',
       'growth_day'],
      dtype='object')

In [ ]:
test_(train)

평균 RMSE :0.4636


0.4636

In [ ]:
new_col = ['청색광추정광량*청색 크기차', '실내 일교차*growth_day', 'growth_day*EC관측치', '청색 최소*블루 LED동작강도', '외부 팬*난방 시간*적색 최소']

In [ ]:
train = get_new_col(train, new_col)
test_(train)

평균 RMSE :0.4416


0.4416

In [ ]:
test_new_col = ['청색광추정광량*청색 크기차', '실내 일교차*growth_day', 'growth_day*EC관측치', '청색 최소*블루 LED동작강도', '외부 팬*난방 시간*적색 최소',
       '화이트 LED상태*최근분무량', '최근분무량*냉방부하', '레드 LED동작강도*growth_day*EC관측치', '청색 최소*블루 LED동작강도*냉방부하', '펌프상태*적색광추정광량']

In [ ]:
train = get_new_col(train, test_new_col)
test_(train)

평균 RMSE :0.4145


0.4145

In [ ]:
test_new_col = ['청색광추정광량*청색 크기차', '실내 일교차*growth_day', 'growth_day*EC관측치', '청색 최소*블루 LED동작강도', '외부 팬*난방 시간*적색 최소',
       '화이트 LED상태*최근분무량', '최근분무량*냉방부하', '레드 LED동작강도*growth_day*EC관측치', '청색 최소*블루 LED동작강도*냉방부하', '펌프상태*적색광추정광량',
       '습도_온도*실내 일교차*growth_day', '펌프상태*실내 일교차*growth_day', '냉방 시간*습도_온도','펌프상태*청색 최소*블루 LED동작강도*냉방부하', '백색 크기차*외부 팬*난방 시간*적색 최소']

In [ ]:
train = get_new_col(train, test_new_col)
test_(train)

평균 RMSE :0.3550


0.355

In [ ]:
total_new_col = ['청색광추정광량*청색 크기차', '실내 일교차*growth_day', 'growth_day*EC관측치', '청색 최소*블루 LED동작강도', '외부 팬*난방 시간*적색 최소',
       '화이트 LED상태*최근분무량', '최근분무량*냉방부하', '레드 LED동작강도*growth_day*EC관측치', '청색 최소*블루 LED동작강도*냉방부하', '펌프상태*적색광추정광량',
       '습도_온도*실내 일교차*growth_day', '펌프상태*실내 일교차*growth_day', '냉방 시간*습도_온도','펌프상태*청색 최소*블루 LED동작강도*냉방부하', '백색 크기차*외부 팬*난방 시간*적색 최소',
       '펌프상태*청색 최소*블루 LED동작강도*냉방부하*블루 LED상태']

In [ ]:
train = get_new_col(train, total_new_col)
test_(train)

평균 RMSE :0.3499


0.3499

In [ ]:
train = get_data(train_list)
test = get_data(test_list)
train = get_new_col(train, total_new_col)
test = get_new_col(test, total_new_col)

데이터 스케일링

In [ ]:
result = []
train['label'] = np.log1p(train['label'])

y_train = train['label']
X = train.drop(['label'],axis=1)
X_test = test.drop(['label'],axis=1)
X_col = X.columns

scaler = StandardScaler()

scaler.fit(X)
scaled_X_train  = scaler.transform(X)
scaled_X_train  = pd.DataFrame(scaled_X_train , columns=X_col)
scaled_X_test  = scaler.transform(X_test)
scaled_X_test  = pd.DataFrame(scaled_X_test , columns=X_col)
scaled_X_test

,내부온도관측치,내부습도관측치,CO2관측치,EC관측치,펌프상태,최근분무량,일간누적분무량,냉방상태,난방상태,내부유동팬상태,...,최근분무량*냉방부하,레드 LED동작강도*growth_day*EC관측치,청색 최소*블루 LED동작강도*냉방부하,펌프상태*적색광추정광량,습도_온도*실내 일교차*growth_day,펌프상태*실내 일교차*growth_day,냉방 시간*습도_온도,펌프상태*청색 최소*블루 LED동작강도*냉방부하,백색 크기차*외부 팬*난방 시간*적색 최소,펌프상태*청색 최소*블루 LED동작강도*냉방부하*블루 LED상태
0,0.973812,-0.335361,0.047351,-0.054318,-0.290796,-0.061566,-0.132218,-0.044453,-0.268538,-0.064099,...,-0.056717,-0.124384,-0.251318,-0.420172,-0.631278,-0.238454,-0.329830,-0.242096,-0.096785,-0.242812
1,1.486363,-0.215687,-0.366531,-0.044636,-0.137679,-0.021053,-0.036647,-0.044433,-0.268538,-0.064099,...,-0.009528,-0.124384,-0.251318,-0.420172,-0.614297,-0.231885,-0.027881,-0.242096,-0.096785,-0.242812
2,2.297798,-0.186451,-0.711562,-0.044609,-0.188774,-0.022268,-0.042922,-0.044444,-0.268538,-0.064099,...,0.018454,-0.124384,-0.251318,-0.420172,-0.550018,-0.220243,-0.010224,-0.242096,-0.096785,-0.242812
3,1.872046,-0.207595,-0.566144,-0.044420,-0.171771,-0.023511,-0.034066,-0.044437,-0.204524,-0.064099,...,0.012131,-0.123240,0.166792,-0.348205,-0.487878,-0.198715,-0.014549,0.152752,-0.096785,0.152042
4,1.875743,-0.230817,-0.571558,-0.044339,-0.137763,-0.025763,-0.025901,-0.044435,-0.268538,-0.064099,...,-0.000759,-0.124384,-0.251318,-0.420172,-0.490871,-0.190234,-0.041183,-0.242096,-0.096785,-0.242812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,-0.356605,0.097264,2.078592,-0.059465,-0.171771,-0.019885,-0.021880,-0.044444,-0.268538,-0.064099,...,0.000230,-0.118668,-0.230805,-0.252183,0.551647,-0.024711,0.200654,-0.222723,-0.096785,-0.223344
191,-0.340153,0.093456,1.680324,-0.062055,-0.120760,-0.019885,-0.021994,-0.044444,-0.268538,-0.064099,...,0.000822,-0.121271,-0.230646,-0.220694,0.567720,0.020876,0.194334,-0.218914,-0.096785,-0.219580
192,-0.422025,0.094323,1.763370,-0.064480,-0.171771,-0.019885,-0.011055,-0.044443,-0.268538,-0.064099,...,-0.002224,-0.123860,-0.231425,-0.251871,0.694534,0.004472,0.201976,-0.223309,-0.096785,-0.223906
193,-0.417220,0.094540,1.689644,-0.064951,-0.120760,-0.019885,-0.021926,-0.044444,-0.268538,-0.064099,...,-0.002036,-0.124382,-0.231378,-0.220063,0.787298,0.071018,0.185829,-0.219734,-0.096785,-0.220356


모델생성

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', max_depth = 5)
xg_reg.fit(scaled_X_train, y_train)
pred = xg_reg.predict(scaled_X_test)
for k in pred:
    result = np.hstack([result, k])
result

array([ 0.99123871,  0.48599562,  0.24713942,  0.26390427,  0.2625449 ,
        0.27300376,  0.25360614,  0.2867794 ,  0.2867794 ,  0.29047346,
        0.28121072,  0.27224809,  0.2748611 ,  0.26913667,  0.26784384,
        0.26833716,  0.22619531,  0.15846986,  0.1650956 ,  0.21463415,
        0.16896051,  0.172362  ,  0.16515067,  0.16494748,  0.15113705,
        0.15846354,  0.13455194,  0.09446374, -0.06056458,  0.34347335,
        0.43646967,  0.36293262,  0.28100979,  0.27313799,  0.27682301,
        0.27713579,  0.2851463 ,  0.27987158,  0.27965993,  0.28008872,
        0.25209743,  0.23513412,  0.25118303,  0.25205475,  0.24322248,
        0.20038363,  0.18079504,  0.18343315,  0.21086371,  0.10388836,
        0.06806332,  0.08069554,  0.08674127,  0.0606499 ,  0.05564246,
        0.05568045,  0.05413526,  0.05873084,  0.04828283,  0.01975834,
        0.00676182, -0.00547457, -0.01781404, -0.01245743,  0.56618482,
        0.3722747 ,  0.36997449,  0.28132239,  0.30063459,  0.29

결과값 로그변환 및 압축

In [ ]:
expm1_result = np.expm1(result)
expm1_result = pd.DataFrame(columns=['label'],data=expm1_result)
total_result = []
for case in tqdm(range(1, 7)):
    if case < 10:
        case = '0'+str(case)
    particular_label = pd.read_csv(f'data/test_target/TEST_{case}.csv')
    particular_label['rate'] = list(expm1_result['label'][:len(particular_label)].values)
    expm1_result = expm1_result.drop([i for i in range(len(particular_label))])
    expm1_result = expm1_result.reset_index(drop=True)
    total_result.append(particular_label)
for i in range(len(total_result)):
    total_result[i].to_csv(f'TEST_0{i+1}.csv', index=False)
file_ls = ['TEST_01.csv','TEST_02.csv','TEST_03.csv','TEST_04.csv','TEST_05.csv','TEST_06.csv']
with zipfile.ZipFile("submission.zip", 'w') as my_zip:
    for i in file_ls:
        my_zip.write(i)
    my_zip.close()

100%|██████████| 6/6 [00:00<00:00, 311.23it/s]
